# Greither unit index, a Sagemath implementation


## Definition of $n$ end its factorization

In [51]:
n=(3**2)*7*11
fn = factor(n)
show(fn)

3^2 * 7 * 11

Also we define these simple functions that we will use later to have a simple overview of the quatity used.

In [52]:
def p(i):
    return fn[i-1][0]
def e(i):
    return fn[i-1][1]
def pe(i):
    return p(i)**e(i)

In [53]:
pe(1)

9

## Power set $P_S$

In [6]:
s = len(fn)
S = [ i +1 for i in range(s)]
PS = Subsets(S).list()
PS.remove(Set(S))

In [7]:
PS

[{}, {1}, {2}, {3}, {1, 2}, {1, 3}, {2, 3}]

In [8]:
def nI(I):
    ret = 1
    for i in I:
        ret *= pe(i)
    return ret

In [9]:
nI({1,3})

55

## Definition of $\zeta$ and the group $G_0$

In [10]:
Qn.<z>=CyclotomicField(n)

In [ ]:
G_0 = Qn.galois_group() #loooooong time

In [11]:
G = [ a for a in [1..n//2] if gcd(a,n)==1]

In [12]:
def corr(r):
    if r not in G:
        r = n-r
    return r
def power(x,a):
    r = power_mod(x,a,n)
    return corr(r)

def mol(x,y):
    r = mod(x*y,n)
    return corr(n)

In [46]:
show(G)

[1,
 2,
 3,
 4,
 6,
 8,
 9,
 12,
 13,
 16,
 17,
 18,
 19,
 23,
 24,
 26,
 27,
 29,
 31,
 32,
 34,
 36,
 37,
 38,
 39,
 41,
 43,
 46,
 47,
 48,
 51,
 52,
 53,
 54,
 57,
 58,
 59,
 61,
 62,
 64,
 67,
 68,
 69,
 71,
 72,
 73,
 74,
 76,
 78,
 79,
 81,
 82,
 83,
 86,
 87,
 89,
 92,
 93,
 94,
 96,
 97,
 101,
 102,
 103,
 104,
 106,
 107,
 108,
 109,
 111,
 113,
 114,
 116,
 117,
 118,
 122,
 123,
 124,
 127,
 128,
 129,
 131,
 134,
 136,
 137,
 138,
 139,
 141,
 142,
 144,
 146,
 148,
 149,
 151,
 152,
 153,
 156,
 157,
 158,
 159,
 162,
 163,
 164,
 166,
 167,
 169,
 171,
 172,
 173,
 174,
 177,
 178,
 179,
 181,
 183,
 184,
 186,
 188,
 191,
 192]

# Calculation of the index
First we define the number field $K$ as the Maximal Real subfield of $\mathbb{Q}(\zeta_n)$

In [14]:
zz = z + z.conjugate()

In [15]:
K = NumberField(zz.minpoly(),'a')

We define $\epsilon_i$ knowing that is equal to $\phi(p_i^{e_i})$

In [16]:
def eps(i):
    return euler_phi(pe(i))

In [17]:
def g(i):
    I = K.ideal(p(i))
    return len(I.factor())

For $f_i$ we use the equality $[K:\mathbb{Q}] = \epsilon_i g_i f_i $ since we know the other three elements.  
*Memo*: $[K:\mathbb{Q}] = \phi(n)/2 $

In [18]:
def f(i):
    return euler_phi(n)/(2*g(i)*eps(i))

So we have that:

In [19]:
def i_b():
    i_b=1
    for i in S:
        i_b *= (eps(i)**(g(i)-1)) * (f(i)**(2*g(i) -1))
    return i_b

In [ ]:
i = i_b()
i

In [ ]:
show(factor(i))

In [20]:
def i_b_compressed(n):
    fn = factor(n)
    s = len(fn)
    S = [ i +1 for i in range(s)]
    K.<z> = CyclotomicField(n)
    zz = z + z.conjugate()
    K = NumberField(zz.minpoly(),'a')
    ibb=1
    for j in S:
        eps = euler_phi(fn[j-1][0]**fn[j-1][1])
        g = len(K.ideal(fn[j-1][0]).factor())
        f = euler_phi(n)/(2*g*eps)
        ibb *= (eps**(g-1)) * (f**(2*g-1))
    return ibb

In [ ]:
factor(i_b_compressed(3*5*7*11))

# Define the Frobenious morphism
Given $i \in S$ we want to find a lift in $G_0$ of the frobenious morphism:

\begin{alignat*}{2}
		F_i : \mathbb{Q}(\zeta_{n/p_i^{e_i}})^+ &\longrightarrow \: \mathbb{Q}(\zeta_{n/p_i^{e_i}})^+  \\
		\zeta_{n/p_i^{e_i}}  &\longmapsto \: \zeta_{n/p_i^{e_i}} ^ {p_i}
\end{alignat*} 
So we need an element $f$ that sends $\zeta_{n/p_i^{e_i}} \simeq \zeta _n^{p_i^{e_i}}$ in $\zeta _n^{p_i^{e_i+1}}= \zeta _n^{p_i^{e_i} p_i}$

In [21]:
def frob(i):
    zi = z^pe(i)
    for f in G:
        if zi^f==zi^p(i):
            return f

In [22]:
def frobhom(i):
    zi = z^pe(i)
    for f in G:
        if zi^f==zi^p(i):
            return Qn.hom([z**f])

In [23]:
frobhom(2)

Ring endomorphism of Cyclotomic Field of order 385 and degree 240
  Defn: z |--> z^62

Now to find the trace elements we need to have the order of the element $f$ in $G_i$.  
We already know that this is the inertia degree of the prime $p_i$, but now we will follow a different approach.

In [28]:
def ford(i):
    Qi.<zi>=CyclotomicField(n/pe(i))
    f = Qi.hom([zi^p(i)])
    o = 1
    zz=zi+zi.conjugate()
    while (not (f**o)(zz)==zz) : 
        o += 1
    return o

In [26]:
[ford2(i)==f(i) for i in S]

[True, True, True]

## Definition of $\beta$ and its evaluation  
**Remark**: we use `f(i)` instead of `ford(i)` because it is faster

In [31]:
def beta0(i : int):
    fi = frobhom(i)
    return [ fi**j for j in range(ford(i))]

In [32]:
vbeta = [beta0(i) for i in S]

For our costruction the only thing we need is $\zeta ^{ \beta(I)}$, that we can evaluate starting from $\zeta ^{ \beta(I)}$ using that $\beta(I) = \prod_{i \in S} \beta(i)$ and $\zeta^{\gamma \delta} = (\zeta^\gamma )^\delta$

In [33]:
def valbeta0(base,i):
    v = 1
    for vf in vbeta[i-1]:
        v *= vf(base)
    return v

In [34]:
def valbeta(base,I):
    if I.is_empty():
        return base
    for i in I:
        base = valbeta0(base,i)
    return base

# Calculation of the generators

In [35]:
def zbeta():
    ret = 1
    for I in PS:
        zI= 1 - z**nI(I)
        ret *= valbeta(zI,I)
    return ret

In [42]:
zb = zbeta()

In [43]:
zb

-44114542003514867154026550977274703476279511493763267171427172262359207053924560546875*z^239 - 88229084007029734308053101954549406952559022987526534342854344524718414107849121093750*z^238 - 88229084007029734308053101954549406952559022987526534342854344524718414107849121093750*z^236 + 44114542003514867154026550977274703476279511493763267171427172262359207053924560546875*z^233 + 44114542003514867154026550977274703476279511493763267171427172262359207053924560546875*z^232 + 44114542003514867154026550977274703476279511493763267171427172262359207053924560546875*z^231 + 88229084007029734308053101954549406952559022987526534342854344524718414107849121093750*z^229 + 44114542003514867154026550977274703476279511493763267171427172262359207053924560546875*z^228 + 88229084007029734308053101954549406952559022987526534342854344524718414107849121093750*z^227 + 88229084007029734308053101954549406952559022987526534342854344524718414107849121093750*z^225 - 4411454200351486715402655097727470347627951149376

In [37]:
def zdbeta(a,I):
    d = (1-a)*nI(I)/2
    valbeta(z^d,I)

In [38]:
def zdbeta(a):
    ret = 1
    d= (1-a)/2
    for I in PS:
        ret *= valbeta(z^(d*nI(I)),I)
    return ret

In [39]:
def xibeta(a):
    sa = Qn.hom([z**a])
    return zdbeta(a)*sa(zb)/zb

In [45]:
xibeta(12).conjugate()==xibeta(12)

True